http://results.openaddresses.io/

In [1]:
import os, zipfile, csv

In [2]:
datadir = os.path.join("..", "..", "..", "..", "Data")
filename = os.path.join(datadir, "openaddr-collected-us_midwest.zip")
zf = zipfile.ZipFile(filename)
for info in zf.filelist:
    if info.filename.startswith("us/il"):
        print(info)

<ZipInfo filename='us/il/shelby.vrt' compress_type=deflate filemode='?rw-------' file_size=336 compress_size=197>
<ZipInfo filename='us/il/shelby.csv' compress_type=deflate filemode='-rw-------' file_size=417777 compress_size=126318>
<ZipInfo filename='us/il/city_of_west_peoria.vrt' compress_type=deflate filemode='?rw-------' file_size=362 compress_size=211>
<ZipInfo filename='us/il/city_of_west_peoria.csv' compress_type=deflate filemode='-rw-------' file_size=429901 compress_size=125392>
<ZipInfo filename='us/il/mcdonough.vrt' compress_type=deflate filemode='?rw-------' file_size=342 compress_size=201>
<ZipInfo filename='us/il/mcdonough.csv' compress_type=deflate filemode='-rw-------' file_size=1151133 compress_size=359625>
<ZipInfo filename='us/il/schuyler.vrt' compress_type=deflate filemode='?rw-------' file_size=340 compress_size=201>
<ZipInfo filename='us/il/schuyler.csv' compress_type=deflate filemode='-rw-------' file_size=576995 compress_size=234171>
<ZipInfo filename='us/il/ma

In [3]:
import pandas as pd
with zf.open('us/il/cook.csv') as f:
    frame = pd.read_csv(f, low_memory=False)

In [4]:
frame.head()

,LON,LAT,NUMBER,STREET,UNIT,CITY,DISTRICT,REGION,POSTCODE,ID,HASH
0,-88.190438,42.074161,4589,WEST HIGGINS ROAD,NaN,Hoffman Estates,NaN,NaN,60192.0,NaN,7e424765c7c8f0c0
1,-88.135805,42.153453,125,SOUTH HOUGH STREET,NaN,Barrington,NaN,NaN,60010.0,NaN,099b736c8dfa0f1f
2,-88.122677,42.147437,104,QUEENS COVE,NaN,Barrington,NaN,NaN,60010.0,NaN,a924636168daf4ed
3,-88.122906,42.092878,1732,WEST ALGONQUIN ROAD,NaN,Hoffman Estates,NaN,NaN,60120.0,NaN,dc1563d64be36394
4,-88.123053,42.092878,1736,WEST ALGONQUIN ROAD,NaN,Hoffman Estates,NaN,NaN,60120.0,NaN,9efa31fc81d103e7


In [5]:
import shapely.geometry
import geopandas as gpd
f = gpd.GeoDataFrame(frame[frame.LON.notnull() & frame.STREET.notnull()])

geo = []
for lo, la in zip(f.LON, f.LAT):
    geo.append( shapely.geometry.Point(lo, la) )
f.geometry = geo

f.crs = {"init" : "epsg:4326"}
f = f.drop(["LON", "LAT", "DISTRICT", "REGION", "ID", "HASH"], axis=1)
f.head()

,NUMBER,STREET,UNIT,CITY,POSTCODE,geometry
0,4589,WEST HIGGINS ROAD,NaN,Hoffman Estates,60192.0,POINT (-88.19043809999999 42.0741614)
1,125,SOUTH HOUGH STREET,NaN,Barrington,60010.0,POINT (-88.1358051 42.1534529)
2,104,QUEENS COVE,NaN,Barrington,60010.0,POINT (-88.1226772 42.14743720000001)
3,1732,WEST ALGONQUIN ROAD,NaN,Hoffman Estates,60120.0,POINT (-88.1229055 42.0928783)
4,1736,WEST ALGONQUIN ROAD,NaN,Hoffman Estates,60120.0,POINT (-88.1230529 42.0928784)


In [6]:
f.to_file("Chicago_Addresses")